In [2]:
import sys
import pandas as pd
import numpy as np
from typing import Any
from scipy.stats import linregress, ttest_1samp

%load_ext autoreload
%autoreload 2
# sys.path.append("D:/Github/note/module")                        # for windows
sys.path.append("/Users/xinc./Documents/GitHub/note/module")    # for mac
from get_info_FinMind import GetInfoFinMind
FinMind_fetcher = GetInfoFinMind()
from plot_func import plot, plot_scatter, plot_df_columns, plot_pdf, plot_dropped_positions, plot_sequence
from performance_func import summarize_performance, mean_ttest

start_date = pd.to_datetime("2020-01-01", format = "%Y-%m-%d")
end_date = pd.to_datetime("2025-09-25", format = "%Y-%m-%d")

# download data

In [157]:
# TXO
op_df = FinMind_fetcher.get_option_daily("TXO", start_date, end_date, trading_session = "all")
op_df.reset_index(drop = True, inplace = True)
op_df_position = op_df[op_df["trading_session"] == "position"].copy()

# TXOa
op_df_after = op_df[op_df["trading_session"] == "after_market"].copy()
op_df_position.to_pickle("../data/TXO.pkl")
op_df_after.to_pickle("../data/TXO_after.pkl")

# TX
tx_df = FinMind_fetcher.init_df("TX", start_date, end_date)
tx_df = FinMind_fetcher.get_future_price()
tx_df.to_csv("../data/TX.csv")

# TXa
txa_df = FinMind_fetcher.init_df("TX", start_date, end_date)
txa_df = FinMind_fetcher.get_future_price(trading_session = "after_market")
txa_df.to_csv("../data/TX_after.csv")

2025-10-21 11:10:47.150 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanOptionDaily, data_id: TXO
2025-10-21 11:12:26.065 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

2025-10-21 11:12:28.603 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping col

# read

In [ ]:
"""read tx, txo"""
tx_df = pd.read_csv("../data/TX.csv")
txa_df = pd.read_csv("../data/TX_after.csv")
op_df = pd.read_pickle("../data/TXO.pkl")
opa_df = pd.read_pickle("../data/TXO_after.pkl")
twii_df = pd.read_pickle("../data/twii.pkl")
twii_df.index = pd.to_datetime(twii_df.index)
twii_df = twii_df.loc[start_date: end_date]

for frame in (tx_df, txa_df):
    frame["Timestamp"] = pd.to_datetime(frame["Timestamp"])
op_df["date"] = pd.to_datetime(op_df["date"])
opa_df["date"] = pd.to_datetime(opa_df["date"])

def build_atm(option_df: pd.DataFrame, price_map: pd.Series, label: str) -> pd.DataFrame:
    atm = option_df.copy()
    atm[label] = atm["date"].map(price_map)
    atm = atm[atm[label].notna()].copy()
    atm["moneyness_gap"] = (atm["strike_price"] - atm[label]).abs()
    atm = (atm.sort_values(["date", "call_put", "moneyness_gap", "volume", "open_interest"],
                          ascending = [True, True, True, False, False])
              .groupby(["date", "call_put"], as_index = False)
              .head(1)
              .reset_index(drop = True))
    return atm.drop(columns = [label, "moneyness_gap"])

_tx_close_map = tx_df.set_index("Timestamp")["Close"]
_tx_open_map = tx_df.set_index("Timestamp")["Open"]
op_atm_close_df = build_atm(op_df, _tx_close_map, "day_close")
op_atm_open_df = build_atm(op_df, _tx_open_map, "day_open")

_txa_close_map = txa_df.set_index("Timestamp")["Close"]
_txa_open_map = txa_df.set_index("Timestamp")["Open"]
opa_atm_close_df = build_atm(opa_df, _txa_close_map, "night_close")
opa_atm_open_df = build_atm(opa_df, _txa_open_map, "night_open")

# diff scenario

## 夜盤 close - 日盤 open

### put

#### ret

In [60]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] / put_df["prev_night_close"]) - 1, np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [61]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

# normalize cum ret
put_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_cross_ret"] / put_df["cum_demeaned_cross_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [68]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] - put_df["prev_night_close"]), np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [70]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
put_df["cum_cross_ret"] = put_df["cross_ret"].cumsum()
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()


plot(put_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] < 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

### call

#### ret

In [98]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
              .rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] / call_df["prev_night_close"]) - 1, np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset=["cross_ret", "tx_ret"], inplace=True)



In [99]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

# normalize
call_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_cross_ret"] / call_df["cum_demeaned_cross_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [100]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

# get cross ret
call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
            .rename(columns = {"close": "prev_night_close"}),
            left_index = True,
            right_on = "date",
            how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] - call_df["prev_night_close"]), np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset=["cross_ret", "tx_ret"], inplace=True)



In [101]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
call_df["cum_cross_ret"] = call_df["cross_ret"].cumsum()
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

plot(call_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[put_df["gap_days"] < 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[put_df["gap_days"] >= 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

## 夜盤 open - 日盤 open

### put

#### ret

In [16]:
put_open_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_open_df = put_open_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
              .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_open_df["date"] = pd.to_datetime(put_open_df["date"])
position_open = (
    op_df.loc[(op_df["call_put"] == "put"),
              ["date", "strike_price", "open"]]
        .drop_duplicates(["date", "strike_price"])
        .rename(columns = {"open": "day_open"})
)
put_open_df = put_open_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_open_df["prev_night_open"] > 0) & (put_open_df["day_open"] > 0)
put_open_df["cross_ret"] = np.where(mask, (put_open_df["day_open"] / put_open_df["prev_night_open"]) - 1, np.nan)

# get date delta
put_open_df["gap_days"] = put_open_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
put_open_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_open_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_open_df = put_open_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_open_df.dropna(subset=["cross_ret", "tx_ret"], inplace=True)



In [17]:
put_open_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross_ret ret
mean = put_open_df["cross_ret"].mean()
put_open_df["demeaned_cross_ret"] = put_open_df["cross_ret"] - mean
put_open_df["cum_demeaned_cross_ret"] = put_open_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_open_df["tx_ret"].mean()
put_open_df["demeaned_tx_ret"] = put_open_df["tx_ret"] - mean
put_open_df["cum_demeaned_tx_ret"] = put_open_df["demeaned_tx_ret"].cumsum()

# normalize
put_open_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_open_df["cum_demeaned_cross_ret"] / put_open_df["cum_demeaned_cross_ret"].std(),
    put_open_df["cum_demeaned_tx_ret"] / put_open_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_open_df.loc[put_open_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_open_df.loc[put_open_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [104]:
put_open_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_open_df = put_open_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
                .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_open_df["date"] = pd.to_datetime(put_open_df["date"])
position_open = (
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]]
    .drop_duplicates(["date", "strike_price"])
    .rename(columns = {"open": "day_open"})
)
put_open_df = put_open_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_open_df["prev_night_open"] > 0) & (put_open_df["day_open"] > 0)
put_open_df["cross_ret"] = np.where(mask, (put_open_df["day_open"] - put_open_df["prev_night_open"]), np.nan)

# get date delta
put_open_df["gap_days"] = put_open_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
put_open_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_open_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_open_df = put_open_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_open_df.dropna(subset=["cross_ret", "tx_ret"], inplace=True)



In [105]:
put_open_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross_ret ret
put_open_df["cum_cross_ret"] = put_open_df["cross_ret"].cumsum()
mean = put_open_df["cross_ret"].mean()
put_open_df["demeaned_cross_ret"] = put_open_df["cross_ret"] - mean
put_open_df["cum_demeaned_cross_ret"] = put_open_df["demeaned_cross_ret"].cumsum()

# cum tx ret
put_open_df["cum_tx_ret"] = put_open_df["tx_ret"].cumsum()
mean = put_open_df["tx_ret"].mean()
put_open_df["demeaned_tx_ret"] = put_open_df["tx_ret"] - mean
put_open_df["cum_demeaned_tx_ret"] = put_open_df["demeaned_tx_ret"].cumsum()

plot(put_open_df.loc[put_open_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_open_df.loc[put_open_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_open_df.loc[put_open_df["gap_days"] < 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_open_df.loc[put_open_df["gap_days"] >= 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

### call

#### ret

In [19]:
call_open_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_open_df = call_open_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_open_df["date"] = pd.to_datetime(call_open_df["date"])
call_open_df.sort_values("date", inplace = True)
call_open_df = call_open_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_open_mask = (call_open_df["prev_night_open"] > 0) & (call_open_df["day_open"] > 0)
call_open_df["open_to_open_ret"] = np.where(
    call_open_mask,
    (call_open_df["day_open"] / call_open_df["prev_night_open"]) - 1,
    np.nan
)
call_open_df["gap_days"] = call_open_df["date"].diff().dt.days
call_open_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
call_open_df = call_open_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_open_df.dropna(subset=["open_to_open_ret", "tx_ret"], inplace=True)
call_open_df




,date,strike_price,prev_night_open,day_open,open_to_open_ret,gap_days,tx_ret
0,2020-01-02,12000.0,26.0,45.0,0.730769,NaN,0.004001
1,2020-01-03,12150.0,42.0,78.0,0.857143,1.0,0.006695
2,2020-01-06,12050.0,84.0,46.5,-0.446429,3.0,-0.006202
3,2020-01-07,12000.0,21.0,33.0,0.571429,1.0,0.004775
4,2020-01-08,11900.0,17.5,2.9,-0.834286,1.0,-0.011713
...,...,...,...,...,...,...,...
1390,2025-09-19,25850.0,114.0,141.0,0.236842,1.0,0.005563
1391,2025-09-22,25750.0,116.0,128.0,0.103448,3.0,0.003467
1392,2025-09-23,26050.0,34.5,87.0,1.521739,1.0,0.007501
1393,2025-09-24,26400.0,28.0,48.0,0.714286,1.0,0.004303


In [20]:
call_open_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = call_open_df["open_to_open_ret"].mean()
call_open_df["demeaned_open_to_open_ret"] = call_open_df["open_to_open_ret"] - mean
call_open_df["cum_demeaned_open_to_open_ret"] = call_open_df["demeaned_open_to_open_ret"].cumsum()

mean = call_open_df["tx_ret"].mean()
call_open_df["demeaned_tx_ret"] = call_open_df["tx_ret"] - mean
call_open_df["cum_demeaned_tx_ret"] = call_open_df["demeaned_tx_ret"].cumsum()

call_open_df[["norm_cum_demeaned_open_to_open_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_open_df["cum_demeaned_open_to_open_ret"] / call_open_df["cum_demeaned_open_to_open_ret"].std(),
    call_open_df["cum_demeaned_tx_ret"] / call_open_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_open_df.loc[call_open_df["gap_days"] < 2], ["norm_cum_demeaned_open_to_open_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_open_df.loc[call_open_df["gap_days"] >= 2], ["norm_cum_demeaned_open_to_open_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [108]:
call_open_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_open_df = call_open_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_open_df["date"] = pd.to_datetime(call_open_df["date"])
call_open_df.sort_values("date", inplace = True)
call_open_df = call_open_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_open_mask = (call_open_df["prev_night_open"] > 0) & (call_open_df["day_open"] > 0)
call_open_df["open_to_open_ret"] = np.where(
    call_open_mask,
    (call_open_df["day_open"] - call_open_df["prev_night_open"]),
    np.nan
)
call_open_df["gap_days"] = call_open_df["date"].diff().dt.days
call_open_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
call_open_df = call_open_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_open_df.dropna(subset=["open_to_open_ret", "tx_ret"], inplace=True)



In [109]:
call_open_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum open_to_open ret
call_open_df["cum_open_to_open_ret"] = call_open_df["open_to_open_ret"].cumsum()
mean = call_open_df["open_to_open_ret"].mean()
call_open_df["demeaned_open_to_open_ret"] = call_open_df["open_to_open_ret"] - mean
call_open_df["cum_demeaned_open_to_open_ret"] = call_open_df["demeaned_open_to_open_ret"].cumsum()

# cum tx ret
call_open_df["cum_tx_ret"] = call_open_df["tx_ret"].cumsum()
mean = call_open_df["tx_ret"].mean()
call_open_df["demeaned_tx_ret"] = call_open_df["tx_ret"] - mean
call_open_df["cum_demeaned_tx_ret"] = call_open_df["demeaned_tx_ret"].cumsum()

plot(call_open_df.loc[call_open_df["gap_days"] < 2], ["cum_open_to_open_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_open_df.loc[call_open_df["gap_days"] >= 2], ["cum_open_to_open_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_open_df.loc[call_open_df["gap_days"] < 2], ["cum_demeaned_open_to_open_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_open_df.loc[call_open_df["gap_days"] >= 2], ["cum_demeaned_open_to_open_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

## 放假前 日盤 open - 日盤 close

### put

#### ret

In [22]:
"""get date gap days"""
op_atm_put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

op_atm_put_df["date"] = pd.to_datetime(op_atm_put_df["date"])
op_atm_put_df.sort_values("date", inplace = True)
op_atm_put_df["gap_days"] = op_atm_put_df["date"].diff().dt.days
op_atm_put_df["gap_days"] = op_atm_put_df["gap_days"].shift(-1)
op_atm_put_df["daily_ret"] = (op_atm_put_df["close"] / op_atm_put_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
op_atm_put_df = op_atm_put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
op_atm_put_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)
op_atm_put_df


,date,option_id,contract_date,strike_price,call_put,open,max,min,close,volume,settlement_price,open_interest,trading_session,gap_days,daily_ret,tx_ret
0,2020-01-02,TXO,202001W1,12100.0,put,63.0,70.0,0.1,0.1,112084,0.0,38913,position,1.0,-0.998413,0.004816
1,2020-01-03,TXO,202001W2,12100.0,put,25.0,116.0,21.0,56.0,57213,56.0,9210,position,3.0,1.240000,-0.007718
2,2020-01-06,TXO,202001W2,11950.0,put,27.5,42.0,18.0,40.5,48310,40.5,18548,position,1.0,0.472727,-0.005575
3,2020-01-07,TXO,202001W2,11850.0,put,4.2,49.5,3.3,18.5,81273,18.5,17796,position,1.0,3.404762,-0.010338
4,2020-01-08,TXO,202001W2,11800.0,put,61.0,111.0,0.1,0.1,100648,0.0,29030,position,1.0,-0.998361,0.005116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2025-09-19,TXO,202509W4,25650.0,put,108.0,176.0,104.0,165.0,3404,165.0,892,position,3.0,0.527778,-0.006925
1391,2025-09-22,TXO,202509W4,25900.0,put,210.0,265.0,118.0,129.0,4096,129.0,939,position,1.0,-0.385714,0.005241
1392,2025-09-23,TXO,202509W4,26250.0,put,242.0,255.0,77.0,84.0,6505,84.0,863,position,1.0,-0.652893,0.008251
1393,2025-09-24,TXO,202509W4,26250.0,put,26.5,203.0,1.0,103.0,32735,0.0,1897,position,1.0,2.886792,-0.005270


In [23]:
op_atm_put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = op_atm_put_df["daily_ret"].mean()
op_atm_put_df["demeaned_daily_ret"] = op_atm_put_df["daily_ret"] - mean
op_atm_put_df["cum_demeaned_daily_ret"] = op_atm_put_df["demeaned_daily_ret"].cumsum()

mean = op_atm_put_df["tx_ret"].mean()
op_atm_put_df["demeaned_tx_ret"] = op_atm_put_df["tx_ret"] - mean
op_atm_put_df["cum_demeaned_tx_ret"] = op_atm_put_df["demeaned_tx_ret"].cumsum()

op_atm_put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    op_atm_put_df["cum_demeaned_daily_ret"] / op_atm_put_df["cum_demeaned_daily_ret"].std(),
    op_atm_put_df["cum_demeaned_tx_ret"] / op_atm_put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [112]:
"""get date gap days"""
op_atm_put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

op_atm_put_df["date"] = pd.to_datetime(op_atm_put_df["date"])
op_atm_put_df.sort_values("date", inplace = True)
op_atm_put_df["gap_days"] = op_atm_put_df["date"].diff().dt.days
op_atm_put_df["gap_days"] = op_atm_put_df["gap_days"].shift(-1)
op_atm_put_df["daily_ret"] = (op_atm_put_df["close"] - op_atm_put_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
op_atm_put_df = op_atm_put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
op_atm_put_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)



In [113]:
op_atm_put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum daily ret
op_atm_put_df["cum_daily_ret"] = op_atm_put_df["daily_ret"].cumsum()
mean = op_atm_put_df["daily_ret"].mean()
op_atm_put_df["demeaned_daily_ret"] = op_atm_put_df["daily_ret"] - mean
op_atm_put_df["cum_demeaned_daily_ret"] = op_atm_put_df["demeaned_daily_ret"].cumsum()

# cum tx ret
op_atm_put_df["cum_tx_ret"] = op_atm_put_df["tx_ret"].cumsum()
mean = op_atm_put_df["tx_ret"].mean()
op_atm_put_df["demeaned_tx_ret"] = op_atm_put_df["tx_ret"] - mean
op_atm_put_df["cum_demeaned_tx_ret"] = op_atm_put_df["demeaned_tx_ret"].cumsum()

plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] < 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(op_atm_put_df.loc[op_atm_put_df["gap_days"] >= 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

### call

#### ret

In [25]:
"""get date gap days"""
op_atm_call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

op_atm_call_df["date"] = pd.to_datetime(op_atm_call_df["date"])
op_atm_call_df.sort_values("date", inplace = True)
op_atm_call_df["gap_days"] = op_atm_call_df["date"].diff().dt.days
op_atm_call_df["gap_days"] = op_atm_call_df["gap_days"].shift(-1)
op_atm_call_df["daily_ret"] = (op_atm_call_df["close"] / op_atm_call_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
op_atm_call_df = op_atm_call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
op_atm_call_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)
op_atm_call_df



,date,option_id,contract_date,strike_price,call_put,open,max,min,close,volume,settlement_price,open_interest,trading_session,gap_days,daily_ret,tx_ret
0,2020-01-02,TXO,202001W1,12100.0,call,5.7,21.0,1.2,2.0,134030,0.0,16210,position,1.0,-0.649123,0.004816
1,2020-01-03,TXO,202001W2,12100.0,call,109.0,122.0,31.5,52.0,52247,52.0,8766,position,3.0,-0.522936,-0.007718
2,2020-01-06,TXO,202001W2,11950.0,call,119.0,119.0,47.0,47.5,16541,48.5,4983,position,1.0,-0.600840,-0.005575
3,2020-01-07,TXO,202001W2,11850.0,call,150.0,165.0,28.0,46.0,55996,46.0,8700,position,1.0,-0.693333,-0.010338
4,2020-01-08,TXO,202001W2,11800.0,call,15.0,98.0,12.0,39.5,121111,0.0,3741,position,1.0,1.633333,0.005116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2025-09-19,TXO,202509W4,25650.0,call,276.0,276.0,158.0,167.0,1440,167.0,539,position,3.0,-0.394928,-0.006925
1391,2025-09-22,TXO,202509W4,25900.0,call,63.0,128.0,40.0,99.0,18975,99.0,3500,position,1.0,0.571429,0.005241
1392,2025-09-23,TXO,202509W4,26250.0,call,25.0,136.0,17.5,86.0,21424,86.0,2751,position,1.0,2.440000,0.008251
1393,2025-09-24,TXO,202509W4,26250.0,call,136.0,143.0,0.1,0.2,42946,0.0,6890,position,1.0,-0.998529,-0.005270


In [26]:
op_atm_call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = op_atm_call_df["daily_ret"].mean()
op_atm_call_df["demeaned_daily_ret"] = op_atm_call_df["daily_ret"] - mean
op_atm_call_df["cum_demeaned_daily_ret"] = op_atm_call_df["demeaned_daily_ret"].cumsum()

mean = op_atm_call_df["tx_ret"].mean()
op_atm_call_df["demeaned_tx_ret"] = op_atm_call_df["tx_ret"] - mean
op_atm_call_df["cum_demeaned_tx_ret"] = op_atm_call_df["demeaned_tx_ret"].cumsum()

op_atm_call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    op_atm_call_df["cum_demeaned_daily_ret"] / op_atm_call_df["cum_demeaned_daily_ret"].std(),
    op_atm_call_df["cum_demeaned_tx_ret"] / op_atm_call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [116]:
op_atm_call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

op_atm_call_df["date"] = pd.to_datetime(op_atm_call_df["date"])
op_atm_call_df.sort_values("date", inplace = True)
op_atm_call_df["gap_days"] = op_atm_call_df["date"].diff().dt.days
op_atm_call_df["gap_days"] = op_atm_call_df["gap_days"].shift(-1)
op_atm_call_df["daily_ret"] = (op_atm_call_df["close"] - op_atm_call_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
op_atm_call_df = op_atm_call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
op_atm_call_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)



In [117]:
op_atm_call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum daily ret
op_atm_call_df["cum_daily_ret"] = op_atm_call_df["daily_ret"].cumsum()
mean = op_atm_call_df["daily_ret"].mean()
op_atm_call_df["demeaned_daily_ret"] = op_atm_call_df["daily_ret"] - mean
op_atm_call_df["cum_demeaned_daily_ret"] = op_atm_call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
op_atm_call_df["cum_tx_ret"] = op_atm_call_df["tx_ret"].cumsum()
mean = op_atm_call_df["tx_ret"].mean()
op_atm_call_df["demeaned_tx_ret"] = op_atm_call_df["tx_ret"] - mean
op_atm_call_df["cum_demeaned_tx_ret"] = op_atm_call_df["demeaned_tx_ret"].cumsum()

plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] < 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(op_atm_call_df.loc[op_atm_call_df["gap_days"] >= 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

### compare to index

In [118]:
tw_df = twii_df.copy()

tw_df["daily_ret"] = (tw_df["close"] / tw_df["open"]) - 1
tw_df["demeaned_daily_ret"] = tw_df["daily_ret"] - tw_df["daily_ret"].mean()

op_atm_put_df.set_index("date", inplace = True)
op_atm_call_df.set_index("date", inplace = True)
op_atm_put_df.index, op_atm_call_df.index, tw_df.index = pd.to_datetime(op_atm_put_df.index), pd.to_datetime(op_atm_call_df.index), pd.to_datetime(tw_df.index)

tw_df = tw_df.reindex(op_atm_put_df.index)
tw_df["cum_demeaned_daily_ret"] = tw_df["demeaned_daily_ret"].cumsum()

op_atm_put_df.reset_index(inplace = True)
op_atm_call_df.reset_index(inplace = True)
tw_df.reset_index(inplace = True)

ret_df = pd.DataFrame({
    "cum_tw_ret": tw_df["cum_demeaned_daily_ret"],
    "cum_put_ret": op_atm_put_df["cum_demeaned_daily_ret"],
    "cum_call_ret": op_atm_call_df["cum_demeaned_daily_ret"]
})

ret_df_norm = ret_df / ret_df.std()
plot_df_columns(ret_df_norm, ["cum_tw_ret", "cum_put_ret", "cum_call_ret"])

## 放假前 夜盤 open - 夜盤 close

### put

#### ret

In [28]:
night_put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
night_put_df["date"] = pd.to_datetime(night_put_df["date"])
night_put_df.sort_values("date", inplace = True)
put_mask = (night_put_df["open"] > 0) & (night_put_df["close"] > 0)
night_put_df["daily_ret"] = np.where(
    put_mask,
    (night_put_df["close"] / night_put_df["open"]) - 1,
    np.nan
)
night_put_df["gap_days"] = night_put_df["date"].diff().dt.days
night_put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
night_put_df = night_put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
night_put_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)
night_put_df


,date,strike_price,open,close,daily_ret,gap_days,tx_ret
0,2020-01-02,12000.0,27.5,10.0,-0.636364,NaN,0.002334
1,2020-01-03,12150.0,85.0,47.0,-0.447059,1.0,0.006034
2,2020-01-06,12050.0,34.0,50.0,0.470588,3.0,-0.004052
3,2020-01-07,12000.0,72.0,39.0,-0.458333,1.0,0.003183
4,2020-01-08,11900.0,45.5,18.5,-0.593407,1.0,0.003455
...,...,...,...,...,...,...,...
1396,2025-09-19,25850.0,230.0,191.0,-0.169565,1.0,0.005213
1397,2025-09-22,25750.0,205.0,155.0,-0.243902,3.0,0.002727
1398,2025-09-23,26050.0,242.0,91.0,-0.623967,1.0,0.007965
1399,2025-09-24,26400.0,187.0,65.0,-0.652406,1.0,0.005826


In [29]:
night_put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = night_put_df["daily_ret"].mean()
night_put_df["demeaned_daily_ret"] = night_put_df["daily_ret"] - mean
night_put_df["cum_demeaned_daily_ret"] = night_put_df["demeaned_daily_ret"].cumsum()

mean = night_put_df["tx_ret"].mean()
night_put_df["demeaned_tx_ret"] = night_put_df["tx_ret"] - mean
night_put_df["cum_demeaned_tx_ret"] = night_put_df["demeaned_tx_ret"].cumsum()

night_put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    night_put_df["cum_demeaned_daily_ret"] / night_put_df["cum_demeaned_daily_ret"].std(),
    night_put_df["cum_demeaned_tx_ret"] / night_put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(night_put_df.loc[night_put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(night_put_df.loc[night_put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [121]:
night_put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
night_put_df["date"] = pd.to_datetime(night_put_df["date"])
night_put_df.sort_values("date", inplace = True)
put_mask = (night_put_df["open"] > 0) & (night_put_df["close"] > 0)
night_put_df["daily_ret"] = np.where(
    put_mask,
    (night_put_df["close"] - night_put_df["open"]),
    np.nan
)
night_put_df["gap_days"] = night_put_df["date"].diff().dt.days
night_put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
night_put_df = night_put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
night_put_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)



In [122]:
night_put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum daily ret
night_put_df["cum_daily_ret"] = night_put_df["daily_ret"].cumsum()
mean = night_put_df["daily_ret"].mean()
night_put_df["demeaned_daily_ret"] = night_put_df["daily_ret"] - mean
night_put_df["cum_demeaned_daily_ret"] = night_put_df["demeaned_daily_ret"].cumsum()

# cum tx ret
night_put_df["cum_tx_ret"] = night_put_df["tx_ret"].cumsum()
mean = night_put_df["tx_ret"].mean()
night_put_df["demeaned_tx_ret"] = night_put_df["tx_ret"] - mean
night_put_df["cum_demeaned_tx_ret"] = night_put_df["demeaned_tx_ret"].cumsum()

plot(night_put_df.loc[night_put_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(night_put_df.loc[night_put_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(night_put_df.loc[night_put_df["gap_days"] < 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(night_put_df.loc[night_put_df["gap_days"] >= 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

### call

#### ret

In [31]:
night_call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
night_call_df["date"] = pd.to_datetime(night_call_df["date"])
night_call_df.sort_values("date", inplace = True)
call_mask = (night_call_df["open"] > 0) & (night_call_df["close"] > 0)
night_call_df["daily_ret"] = np.where(
    call_mask,
    (night_call_df["close"] / night_call_df["open"]) - 1,
    np.nan
)
night_call_df["gap_days"] = night_call_df["date"].diff().dt.days
night_call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
night_call_df = night_call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
night_call_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)
night_call_df



,date,strike_price,open,close,daily_ret,gap_days,tx_ret
0,2020-01-02,12000.0,26.0,30.0,0.153846,NaN,0.002334
1,2020-01-03,12150.0,42.0,73.0,0.738095,1.0,0.006034
2,2020-01-06,12050.0,84.0,64.0,-0.238095,3.0,-0.004052
3,2020-01-07,12000.0,21.0,27.0,0.285714,1.0,0.003183
4,2020-01-08,11900.0,17.5,28.0,0.600000,1.0,0.003455
...,...,...,...,...,...,...,...
1396,2025-09-19,25850.0,114.0,173.0,0.517544,1.0,0.005213
1397,2025-09-22,25750.0,116.0,122.0,0.051724,3.0,0.002727
1398,2025-09-23,26050.0,34.5,92.0,1.666667,1.0,0.007965
1399,2025-09-24,26400.0,28.0,63.0,1.250000,1.0,0.005826


In [32]:
night_call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = night_call_df["daily_ret"].mean()
night_call_df["demeaned_daily_ret"] = night_call_df["daily_ret"] - mean
night_call_df["cum_demeaned_daily_ret"] = night_call_df["demeaned_daily_ret"].cumsum()

mean = night_call_df["tx_ret"].mean()
night_call_df["demeaned_tx_ret"] = night_call_df["tx_ret"] - mean
night_call_df["cum_demeaned_tx_ret"] = night_call_df["demeaned_tx_ret"].cumsum()

night_call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    night_call_df["cum_demeaned_daily_ret"] / night_call_df["cum_demeaned_daily_ret"].std(),
    night_call_df["cum_demeaned_tx_ret"] / night_call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(night_call_df.loc[night_call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(night_call_df.loc[night_call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [39]:
night_call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
night_call_df["date"] = pd.to_datetime(night_call_df["date"])
night_call_df.sort_values("date", inplace = True)
call_mask = (night_call_df["open"] > 0) & (night_call_df["close"] > 0)
night_call_df["daily_ret"] = np.where(
    call_mask,
    (night_call_df["close"] - night_call_df["open"]),
    np.nan
)
night_call_df["gap_days"] = night_call_df["date"].diff().dt.days
night_call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
night_call_df = night_call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
night_call_df.dropna(subset=["daily_ret", "tx_ret"], inplace=True)



In [41]:
night_call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum daily ret
night_call_df["cum_daily_ret"] = night_call_df["daily_ret"].cumsum()
mean = night_call_df["daily_ret"].mean()
night_call_df["demeaned_daily_ret"] = night_call_df["daily_ret"] - mean
night_call_df["cum_demeaned_daily_ret"] = night_call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
night_call_df["cum_tx_ret"] = night_call_df["tx_ret"].cumsum()
mean = night_call_df["tx_ret"].mean()
night_call_df["demeaned_tx_ret"] = night_call_df["tx_ret"] - mean
night_call_df["cum_demeaned_tx_ret"] = night_call_df["demeaned_tx_ret"].cumsum()

plot(night_call_df.loc[night_call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
subset = night_call_df.loc[night_call_df["gap_days"] >= 2, ["gap_days", "cum_daily_ret", "cum_tx_ret"]].copy()
subset["cum_daily_ret"] += 1500
subset["cum_tx_ret"] += 1500
plot(subset, ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(night_call_df.loc[night_call_df["gap_days"] < 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(night_call_df.loc[night_call_df["gap_days"] >= 2], ["cum_demeaned_daily_ret", "cum_demeaned_tx_ret"], ry = "gap_days")


# backtest

## benchmark

In [51]:
short_call_df = op_atm_df.loc[op_atm_df["call_put"] == "call"].copy()


short_call_df

,date,option_id,contract_date,strike_price,call_put,open,max,min,close,volume,settlement_price,open_interest,trading_session
0,2020-01-02,TXO,202001W1,12100.0,call,5.7,21.0,1.2,2.0,134030,0.0,16210,position
2,2020-01-03,TXO,202001W2,12100.0,call,109.0,122.0,31.5,52.0,52247,52.0,8766,position
4,2020-01-06,TXO,202001W2,11950.0,call,119.0,119.0,47.0,47.5,16541,48.5,4983,position
6,2020-01-07,TXO,202001W2,11850.0,call,150.0,165.0,28.0,46.0,55996,46.0,8700,position
8,2020-01-08,TXO,202001W2,11800.0,call,15.0,98.0,12.0,39.5,121111,0.0,3741,position
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780,2025-09-19,TXO,202509W4,25650.0,call,276.0,276.0,158.0,167.0,1440,167.0,539,position
2782,2025-09-22,TXO,202509W4,25900.0,call,63.0,128.0,40.0,99.0,18975,99.0,3500,position
2784,2025-09-23,TXO,202509W4,26250.0,call,25.0,136.0,17.5,86.0,21424,86.0,2751,position
2786,2025-09-24,TXO,202509W4,26250.0,call,136.0,143.0,0.1,0.2,42946,0.0,6890,position


賣方會卡保證金，所以utility會少一點，所以要考慮保證金使用的比例
看call的，報酬金額 / 保證金（市值）

先測整段連續再分放假